In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools as iter_t
import time

retail_data = [i.strip().split() for i in open("retail.dat").readlines()]

retail_data = pd.DataFrame(retail_data)

# allocate a limited number of rows, for testing

i = len(retail_data)
amt = 0.2
retail_data = retail_data[:int(i*amt)]

retail_data.reset_index(inplace=True, drop=True)

print(retail_data.shape)
retail_data

(17632, 76)


,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0,1,2,3,4,5,6,7,8,9,...,None,None,None,None,None,None,None,None,None,None
1,30,31,32,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,33,34,35,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,36,37,38,39,40,41,42,43,44,45,...,None,None,None,None,None,None,None,None,None,None
4,38,39,47,48,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17627,38,170,200,237,431,1872,3759,6211,6335,6386,...,None,None,None,None,None,None,None,None,None,None
17628,9,39,48,101,301,445,549,565,938,1010,...,None,None,None,None,None,None,None,None,None,None
17629,200,2188,3369,4398,4401,7491,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
17630,32,39,48,128,209,438,441,722,740,783,...,None,None,None,None,None,None,None,None,None,None


## Construct Candidate Set

In [84]:
# get unique items
# construct array of all items
# slow
items = []
for i in range(retail_data.shape[1]):
    for j in range(retail_data.shape[0]):
        items.append(retail_data[i][j])

In [85]:
# get unique items
unique_items = list(set(items))
#print(unique_items)
#print(items)

In [86]:
# get item frequency
# construct candidate set
# slow
C1 = []
for val in unique_items:
    C1.append((val, items.count(val)))
    
C1 = pd.DataFrame(C1, columns=[0, "freq"])
C1 = C1.dropna()

In [81]:
# items array is large and no longer needed, get rid of it
del(items)

In [87]:
L1 = C1[C1["freq"] >= SUPPORT]
L1

,0,freq
34,237,3
46,170,3
101,110,3
137,41,24
185,225,4
216,179,3
220,161,3
222,36,8
296,89,4
304,79,3


## Construct C2

In [89]:
C2         = iter_t.combinations(L1[0], 2) # ezpz thanks to itertools
C2         = pd.DataFrame([i for i in C2])      # unroll the generator
C2["freq"] = np.zeros(len(C2[0]), dtype=int)    # create an array of zeros to append to the C2

C2.is_copy = False

# get the frequency of pairs in the items array
# slow
"""
for i in range(retail_data.shape[0]):
    for C2_index in range(C2.shape[0]):
        pair = C2.loc[C2_index]
        _0 = False # keeps track of if the items in the pairs are in that row
        _1 = False
        for j in range(retail_data.shape[1]):
            if retail_data[i][j] == pair[0]:
                _0 = True
            if retail_data[i][j] == pair[1]:
                _1 = True
            if _0 and _1:
                C2["freq"][C2_index] += 1
                break #>
"""
# still slow but vastly better
for i in range(retail_data.shape[0]):
    for j in range(C2.shape[0]):
        if set(C2.loc[j][0:2].to_numpy(dtype=int)).issubset(set(retail_data.loc[i].dropna().to_numpy(dtype=int))):
            C2.loc[j, "freq"] += 1

C2

,0,1,freq
0,237,170,0
1,237,110,0
2,237,41,1
3,237,225,0
4,237,179,0
...,...,...,...
320,258,105,0
321,258,186,1
322,152,105,0
323,152,186,0


In [90]:
L2 = C2[C2["freq"] >= SUPPORT]
L2

,0,1,freq
13,237,39,3
37,170,39,3
43,170,38,3
44,170,48,3
49,110,41,3
75,41,36,3
80,41,32,4
82,41,39,16
88,41,38,10
89,41,48,13


## Construct C3

In [91]:
C3         = iter_t.combinations(set(np.append(L2[0].to_numpy(dtype=int), L2[1].to_numpy(dtype=int))), 3)
C3         = pd.DataFrame([i for i in C3])
C3["freq"] = np.zeros(len(C3[0]), dtype=int)

C3.is_copy = False

for i in range(retail_data.shape[0]):
    for j in range(C3.shape[0]):
        if set(C3.loc[j][0:3].to_numpy(dtype=int)).issubset(set(retail_data.loc[i].dropna().to_numpy(dtype=int))):
            C3.loc[j, "freq"] += 1
            
C3

,0,1,2,freq
0,32,65,36,0
1,32,65,37,0
2,32,65,38,0
3,32,65,39,0
4,32,65,41,0
...,...,...,...,...
675,48,152,89,0
676,242,147,152,0
677,242,147,89,0
678,242,152,89,0


In [92]:
L3 = C3[C3["freq"] >= SUPPORT]
L3

,0,1,2,freq
42,32,38,39,3
49,32,38,48,3
54,32,39,41,3
60,32,39,48,6
70,32,41,48,3
165,65,39,48,3
238,36,38,39,6
239,36,38,41,3
245,36,38,48,4
394,38,39,41,6


Note: Creation of candidate sets is supposed to continue until there are no more tuples with support. This can be put into a function?

## Strong Association

In [93]:
final_1 = L1.copy()
final_2 = L2.copy()
final_3 = L3.copy()

final_1["item"] = L1[0]
final_1         = final_1.drop(labels=0, axis=1)
final_2["item"] = list(zip(L2[0], L2[1]))
final_2         = final_2.drop(labels=[0, 1], axis=1)
final_3["item"] = list(zip(L3[0], L3[1], L3[2]))
final_3         = final_3.drop(labels=[0, 1, 2], axis=1)

final = final_1.append(final_2).append(final_3)
final

,freq,item
34,3,237
46,3,170
101,3,110
137,24,41
185,4,225
...,...,...
410,5,"(38, 41, 48)"
419,3,"(38, 170, 48)"
465,11,"(39, 41, 48)"
474,3,"(39, 170, 48)"


## Turning it into a Function

In [2]:
def apriori(data, min_support=0.05, min_confidence=0.10):
    def n_candidate(prev_L, n):
        names_helper = np.array([], dtype=int)
        for i in range(n-1):
            names_helper = np.append(names_helper, prev_L[i].to_numpy(dtype=int))

        candidate         = iter_t.combinations(set(names_helper), n)
        candidate         = pd.DataFrame([i for i in candidate])
        candidate["freq"] = np.zeros(len(candidate[0]), dtype=int)

        candidate.is_copy = False

        for i in range(data.shape[0]):
            for j in range(candidate.shape[0]):
                if set(candidate.loc[j][0:n].to_numpy(dtype=int)).issubset(set(data.loc[i].dropna().to_numpy(dtype=int))):
                    candidate.loc[j, "freq"] += 1
        return candidate
    
    # get unique items
    # construct array of all items
    items = []
    for i in range(data.shape[1]):
        for j in range(data.shape[0]):
            items.append(data[i][j])
    unique_items = list(set(items))
    
    # construct candidate sets
    C = []
    L = []
    
    C1 = []
    for val in unique_items:
        C1.append((val, items.count(val)))
    
    total_transactions = len(data)

    del(items)
    
    C1 = pd.DataFrame(C1, columns=[0, "freq"], dtype=int)
    C1 = C1.dropna()
    
    C1["conf"] = np.ones(len(C1[0]), dtype=float)
    
    C1["sup"]  = C1["freq"] / total_transactions
    
    L1 = C1[C1["sup"] >= min_support]
    L1 = L1.astype({0: int})
    
    C.append(C1)
    L.append(L1)
    
    def conf(_L, prevL, n):
        # Build confidence
        pd.set_option('mode.chained_assignment', 'warn')
        _L["conf"] = np.zeros(len(_L[0]), dtype=float)

        _L.is_copy = False
        for i in range(len(_L["freq"])):
            oldSup = _L["freq"].iloc[i]                                        #(prevL["freq"][prevL.iloc[:, 0:n] == L_cur[0:n]]).iat[0]
            for j in range(len(prevL[0])):
                if set(prevL.iloc[j, 0:n-1]) == set(_L.iloc[i, 0:n-1]):
                    oldSup = prevL["freq"].iloc[j]
                    break
            _L["conf"].iloc[i] = (_L["freq"].iloc[i] / oldSup)
        
        return _L
        
    
    i = 2
    while True:
        cand         = n_candidate(L[i-2], i)
        cand["sup"]  = cand["freq"] / total_transactions
        L_cur        = cand[cand["sup"] >= min_support]
        
        if len(L_cur[0]) == 0:
            break
        
        L_cur = conf(L_cur, L[i-2], i)
        
        L_cur = L_cur[L_cur["conf"] >= min_confidence]
        
        C.append(cand)
        L.append(L_cur)
        i += 1
    
    return L

In [ ]:
start = time.time()
res   = apriori(retail_data, min_support=0.01, min_confidence=0.0)
end   = time.end()
print(f"Runtime of the program is {end - start}")